Here is my code to find angular separation

In [6]:
#import packages 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy import units as u  
import time

I am using ROSAT data and SDSS list. 

In [8]:
galaxiesFull = pd.read_csv("galaxies_full.csv", sep = ',')  
ROSAT = pd.read_csv("2rxs_lite.txt", delim_whitespace=True)  

RA_Galaxies = ROSAT.iloc[:, 2] #right ascention
DEC_Galaxies = ROSAT.iloc[:, 3] #decllination
PosErr_ROSAT = ROSAT.iloc[:, 4] #position error

RA_ROSAT = ROSAT.iloc[:, 2] #right ascention
DEC_ROSAT = ROSAT.iloc[:, 3] #decllination
PosErr_ROSAT = ROSAT.iloc[:, 4] #position error

In [11]:
start = time.time() #start a time to see how long it takes


count = 0
theta = [] #the angular separation
d1 = [] #declination 1
d2 = [] #declination 2
ra1 = [] #right ascention 1
ra2 = [] #right ascention 2
pos_err1 = [] #position error 1
pos_err2 = [] #position error 2
offset = [] 


for i in range(np.size(DEC_Galaxies)): 
        
        tmp1 = np.where( np.absolute(DEC_Galaxies[i] - DEC_ROSAT) < 1) 
        tmp2 = np.where( np.absolute(RA_Galaxies[i]-RA_ROSAT) * np.cos(DEC_ROSAT*(np.pi/180))  < 1 ) 
        resultArray = (np.intersect1d(tmp1, tmp2))
        
        if np.size(resultArray) ==0: 
                continue
        else: 
                for j in resultArray:

                        theta_rad = np.sin(DEC_Galaxies[i]*np.pi/180)* np.sin(DEC_ROSAT[j]*np.pi/180) + np.cos(DEC_Galaxies[i]*np.pi/180)*np.cos(DEC_ROSAT[j]*np.pi/180)*np.cos(np.absolute(RA_Galaxies[i]-RA_ROSAT[j])*np.pi/180) #returns answer in rad
                        theta_arcsec = np.arccos(theta_rad) * 206265

                        if theta_arcsec < 360: 
                                

                                poserr1 = 1
                                if PosErr_ROSAT[j] < 6:
                                        poserr2 = 6
                                else:
                                        poserr2 = PosErr_ROSAT[j] 
                                confid = theta_arcsec/np.sqrt(poserr1**2 + poserr2**2) #finding theta hat
                                
                                if confid < 4:
                                        count+= 1
                                        theta.append(theta_arcsec)
                                        offset.append(confid)
                                        d1.append(DEC_Galaxies[i])
                                        d2.append(DEC_ROSAT[j])
                                        ra1.append(RA_Galaxies[i])
                                        ra2.append(RA_ROSAT[j])
                                        pos_err1.append(poserr1)
                                        pos_err2.append(poserr2)

print(f'Numpy: {time.time() - start} seconds')

/var/folders/3f/g9qgz71j35z23v4byb9jtltc00031c/T/ipykernel_28513/1861551713.py:27: RuntimeWarning: invalid value encountered in arccos
  theta_arcsec = np.arccos(theta_rad) * 206265


Numpy: 349.2818691730499 seconds


In [12]:
print(count)
print("Writing file")

with open(r'galaxies_ROSAT.txt', 'w') as fp:
        fp.write("Theta\tOffset\tRA-SDSS\tDEC-SDSS\tPosErr-SDSS\tRA-ROSAT\tDEC-ROSAT\tPosErr-ROSAT\n")
        for i in range(count):
                fp.write("%4.1f\t%4.2f\t%12.6f\t%12.6f\t%6.2f\t%12.6f\t%12.6f\t%6.2f\n" %((theta[i]), (offset[i]), (ra1[i]), (d1[i]), (pos_err1[i]), (ra2[i]), (d2[i]), (pos_err2[i]) ))

108183
Writing file


There are 11314 galaxies and it seems thaat 108183 of them are the exact source in this list from SDSS


Now I will try again with SDSS.

In [47]:
searchMatch = np.loadtxt("galaxies_ROSAT.txt" , skiprows = 1)
thetaSearch = searchMatch[:,1]

nonZeroTheta = np.where(thetaSearch != 0)
k = 0
print(count)
print("Writing file")

with open(r'galaxies_ROSAT_nonzero.txt', 'w') as fp:
    fp.write("Theta\tOffset\tRA-SDSS\tDEC-SDSS\tPosErr-SDSS\tRA-ROSAT\tDEC-ROSAT\tPosErr-ROSAT\n")
    for i in range(np.size(nonZeroTheta)):
        k = nonZeroTheta[0][i]
        fp.write("%4.1f\t%4.2f\t%12.6f\t%12.6f\t%6.2f\t%12.6f\t%12.6f\t%6.2f\n" %((theta[k]), (offset[k]), (ra1[k]), (d1[k]), (pos_err1[k]), (ra2[k]), (d2[k]), (pos_err2[k]) ))



108183
Writing file


In [46]:
print(nonZeroTheta[0][1])

2650


In [59]:
start = time.time() #start a time to see how long it takes


Galaxies = pd.read_csv("galaxies_full.csv", sep = ',')  
RA_Galaxies = Galaxies.iloc[:, 1] #right ascention
DEC_Galaxies = Galaxies.iloc[:, 2] #decllination
#PosErr_Galaxies = Galaxies.iloc[:, 4] #position error


SDSS = pd.read_csv("dr7_sara.csv", sep=',', header = 1) #SDSS
RA_SDSS = SDSS.iloc[:, 0] #right ascention
DEC_SDSS = SDSS.iloc[:, 1] #decllination
PosErr_SDSS = SDSS.iloc[:, 2] #position error

print("RA SDSS", np.size(DEC_SDSS))
print("DEC SDSS", np.size(RA_SDSS))
print("RA Galaxies", np.size(RA_Galaxies))
print("DEC Galaxies", np.size(DEC_Galaxies))

count = 0

theta = [] #the angular separation
d1 = []
d2 = []
ra1 = []
ra2 = []
PosErr1 = []
PosErr2 = []
offset = []
k = 0

for i in range(np.size(DEC_SDSS)): 
        
        tmp1 = np.where( np.absolute(DEC_SDSS[i] - DEC_Galaxies) < 1) 
        tmp2 = np.where( np.absolute(RA_SDSS[i]-RA_Galaxies) * np.cos(DEC_Galaxies*(np.pi/180))  < 1 ) 
        resultArray = (np.intersect1d(tmp1, tmp2))

        if np.size(resultArray) ==0: 
                continue
        else: 
                for j in resultArray:

                        theta_rad = np.sin(DEC_SDSS[i]*np.pi/180)* np.sin(DEC_Galaxies[j]*np.pi/180) + np.cos(DEC_SDSS[i]*np.pi/180)*np.cos(DEC_Galaxies[j]*np.pi/180)*np.cos(np.absolute(RA_SDSS[i]-RA_Galaxies[j])*np.pi/180) #returns answer in rad
                        theta_arcsec = np.arccos(theta_rad) * 206265
                        if theta_arcsec < 360: 

                                poserr1 = 6
                                poserr2 = 6
                                """                             
                                if PosErr_Galaxies[j] < 6:
                                        poserr2 = 6

                                else:
                                        poserr2 = PosErr_Galaxies[j] 
                                """
                                confid = theta_arcsec/np.sqrt(poserr1**2 + poserr2**2) #finding theta hat
                                if confid < 4:
                                        count+= 1
                                        theta.append(theta_arcsec)
                                        offset.append(confid)
                                        d1.append(DEC_SDSS[i])
                                        d2.append(DEC_Galaxies[j])
                                        ra1.append(RA_SDSS[i])
                                        ra2.append(RA_Galaxies[j])
                                        PosErr1.append(poserr1)
                                        PosErr2.append(poserr2)
                                        k += 1

                
print(count)

print(f'Numpy: {time.time() - start} seconds')

RA SDSS 9525
DEC SDSS 9525
RA Galaxies 1573
DEC Galaxies 1573


/var/folders/3f/g9qgz71j35z23v4byb9jtltc00031c/T/ipykernel_28513/1568026449.py:44: RuntimeWarning: invalid value encountered in arccos
  theta_arcsec = np.arccos(theta_rad) * 206265


1162
Numpy: 9.511152029037476 seconds


In [60]:
print("Writing file")

with open(r'dr7_galaxies.txt', 'w') as fp:
        fp.write("Theta\tOffset\tRA-SDSS\tDEC-SDSS\tPosErr-SDSS\tRA-Galaxies\tDEC-Galaxies\tPosErr-Galaxies\n")
        for i in range(count):
                fp.write("%4.1f\t%4.2f\t%12.6f\t%12.6f\t%6.2f\t%12.6f\t%12.6f\t%6.2f\n" %((theta[i]), (offset[i]), (ra1[i]), (d1[i]), (PosErr1[i]), (ra2[i]), (d2[i]), (PosErr2[i]) ))

Writing file


It seems that there are no matches with SDSS DR7 release

In [58]:
 Galaxies.iloc[:, 4]

0       0.015561
1       0.016275
2       0.015377
3       0.015220
4       0.014617
          ...   
1568    0.000958
1569    0.001061
1570    0.000954
1571    0.015472
1572    0.000330
Name: Redshift, Length: 1573, dtype: float64